In [95]:
'''
Function:Load the data base and query
Date: 09/20/2017
Author: Yu (Brian) Yao

Input: 
    long: Longitude coordinate in float.
    lat:  Latitude coordinate in float.
    
Output:
    Top three nearest road. 
    
Database file:
    0 - 6:   id, highway, footway, name, tiger:cfcc, lanes, service
    7 - 12:  surface, oneway, amenity, maxspeed, crossing, lit
    13 - 16: lanes:backward, lanes:forward, source, parking
    17 - 19: sidewalk, turn:lanes:backward, turn:lanes:forward
    20 - 24: turn:lanes, destination:ref, barrier, tunnel, water
    25 - 26: hourse, railway
    27:      geometry
    *28:     distance 
    
'''
import sqlite3
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import folium
%matplotlib tk

MAX = 1E-4
MIN = 1E-5
class sql():
    def __init__(self, db_name):
        # connect to database
        self.conn = sqlite3.connect(db_name)
        self.conn.enable_load_extension(True)
        self.conn.load_extension("mod_spatialite")
        
        self.num_roads = 5
        
    def load_tunnel(self):
#         res = self.conn.execute("SELECT tunnel, COUNT(tunnel)\
#                                  FROM ('import.osm_roads') as rd\
#                                  GROUP By tunnel
#                                  ORDER By number DESC")
        res = self.conn.execute("SELECT tunnel, ST_AsText(ST_Transform(rd.geometry,4326))\
                                 FROM ('import.osm_roads') as rd")
        tunnels = res.fetchall()
        print (tunnels[0])
        tunls = []
        geos = []
        for i in range(len(tunnels)):
            tunls.append(tunnels[i][0])
            geos.append(tunnels[i][1])
        tunnel_types = {}
        tunnel_types['tunnel'] = tunls
        tunnel_types['geometry'] = geos
        
        return tunnel_types
        
    def count_highway(self):
        res = self.conn.execute("SELECT highway, COUNT(highway) as number \
                                 FROM ('import.osm_roads') as rd \
                                 GROUP By highway \
                                 ORDER By number DESC")
        highways = res.fetchall()
        res = self.conn.execute("SELECT highway \
                                 FROM ('import.osm_roads') as rd")
        highways = res.fetchall()
        
        fields = []
        for i in range(len(highways)):
            fields.append(highways[i][0])        
        sns.set(style="darkgrid")
        
        road_types = {}
        road_types['class']=fields
        
        
        f = plt.figure(figsize=(15,15.4),dpi=80)
        ax = f.add_subplot(111)
        ax = sns.countplot(y='class', data = road_types, ax=ax)
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)
#         ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
        
    def count_surfaces(self):
        
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface='unpaved' OR \
                                       rd.surface='dirt' OR \
                                       rd.surface='gravel' OR \
                                       rd.surface='ground' OR \
                                       rd.surface='dirt;unpaved' OR \
                                       rd.surface='compacted' OR \
                                       rd.surface='earth' OR \
                                       rd.surface='mud' OR \
                                       rd.surface='sand' OR \
                                       rd.surface='woodchips' OR \
                                       rd.surface='grass' OR \
                                       rd.surface='fine_gravel'")
        unpaved_roads = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface='paved' OR \
                                       rd.surface='asphalt' OR \
                                       rd.surface='concrete' OR \
                                       rd.surface='sett' OR \
                                       rd.surface='concrete:lanes' OR \
                                       rd.surface='concrete:plates' OR \
                                       rd.surface='metal' OR \
                                       rd.surface='wood' OR \
                                       rd.surface='paving_stones' OR \
                                       rd.surface='cobblestone' OR \
                                       rd.surface='pebblestone' OR \
                                       rd.surface='brick'")
        paved_roads = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface=''")
        unlabelled = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd")
        total = res.fetchall()
        print ('# of unpaved roads: ', unpaved_roads[0][0])
        print ('# of paved roads: ', paved_roads[0][0])
        print ('# of unlabeled roads: ', unlabelled[0][0])
        print ('Total # of roads: ', total[0][0])
        
        
        
        # plot coutplot 
        res = self.conn.execute("SELECT rd.surface \
                                 FROM ('import.osm_roads') as rd")
        surface_types = res.fetchall()
        concrete_roads = ['paved', 'asphalt', 'concrete', 'metal', 'wood']
        stone_roads = ['paving_stones', 'cobblestone', 'pebblestone', 'brick', 'sett', 
                       'concrete:lanes', 'concrete:plates', 'paving_stones']
        unpaved_roads = ['unpaved', 'dirt', 'gravel', 'ground', 'dirt;unpaved', 'compacted', 
                         'earth', 'mud', 'sand', 'woodchips', 'grass', 'fine_gravel']
        fields = []
        for i in range(len(surface_types)):
            surface = surface_types[i][0]
            if surface in concrete_roads:
                fields.append('concrete roads')
            elif surface in stone_roads:
                fields.append('stone roads')
            elif surface in unpaved_roads:
                fields.append('unpaved roads')
            else:
                fields.append('unlabelled roads')
        surface_types = {}
        surface_types['types']=fields
        
        sns.set(style="darkgrid")
        f = plt.figure(figsize=(16,8.4),dpi=80)
        ax = f.add_subplot(111)
        ax = sns.countplot(y='types', data = surface_types, ax=ax)
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 1.1, box.height])
#         ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
    
        # add numbers
        for p in ax.patches:
            ax.text(p.get_width() + 10000,
                    p.get_y()+p.get_height()*0.3,
                    '{:1.5f}'.format(p.get_width()/total[0][0]),
                    ha="center", rotation=-90)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)

    def reorder_roads(self, location):
        '''
        Reorder road data by distance 
        '''
        long = location[0]
        lat = location[1]
        '''Find the nearest road''' 
        # data = conn.execute("SELECT *, MIN(DISTANCE(geometry, ST_POINT( -83.702175,42.297199))) as dist \
        #                     FROM OSM_AA_POLYLINES_NEW as osm \
        #                     JOIN ways_tags as wt USING (id)")

        '''Order the roads by distances to the point.'''
        res = self.conn.execute("SELECT *,ST_Distance(osm.geometry, ST_POINT(?,?)) as dist\
                            FROM OSM_AA_POLYLINES_NEW as osm\
                            ORDER BY ST_Distance(osm.geometry, ST_POINT(?,?))",(long,lat,long,lat))
        data = res.fetchall()
        nearest_road = self.check_roads(data)
        
        res = self.conn.execute("SELECT AsText(?), NumPoints(?)",(nearest_road['geometry'], nearest_road['geometry']))
        data = res.fetchall()
        
        return nearest_road
    
    def check_roads(self, data):
        '''
        Given the reordered road data, find the nearest possible road
        '''
        flag = True
        i = 0
        while flag:
            road = self.load_dict(data[i])
            if i == 0 :
                nearest_road = road
            else:
                if road['dist'] - dist_prev < MIN:
                    # Both roads are close
                    if highway_prev == 'footway':
                        # if the closer road is a footway, it may be false
                        # check the next close road
                        if road['highway'] != 'footway':
                            nearest_road = road
                        continue
                elif road['dist'] - dist_prev > MAX:
                    # new road is far
                    return  nearest_road
            dist_prev = road['dist']
            highway_prev = road['highway']
            i += 1
        return nearest_road
            
    def load_dict(self, data):
        road = {}
        road['id'] = data[0]
        road['highway'] = data[1]
        road['footway'] = data[2]
        road['name'] = data[3]
        road['cfcc'] = data[4]
        road['lanes'] = data[5]
        road['service'] = data[6]
        road['surface'] = data[7]
        road['oneway']= data[8]
        road['amenity'] = data[9]
        if data[10] != None:
            road['maxspeed'] = float(data[10][0:2])*0.44704 # mph to m/s
        road['crossing'] = data[11]
        road['lit'] = data[12]
        road['lanes_back'] = data[13]
        road['lanes_forward'] = data[14]
        road['source'] = data[15]
        road['parking'] = data[16]
        road['sidewald'] = data[17]
        road['turn_back'] = data[18]
        road['turn_forward'] = data[19]
        road['turn'] = data[20]
        road['destination'] = data[21]
        road['barrier'] = data[22]
        road['tunnel'] = data[23]
        road['water'] = data[24]
        road['horse'] = data[25]
        road['railway'] = data[26]
        road['geometry'] = data[27]
        road['dist'] = data[28]
        return road
def string2points(road, tunnel, num, m):

    i=11
    coord=''
    points = []
    while road[i]:
        if road[i] == ')':
            lat = float(coord)
            points.append((lat,long))
            break
        if road[i] == ',':
            lat = float(coord)
            points.append((lat,long))
            coord = ''
            i = i+2
            continue
        if road[i]==' ':
            long = float(coord)
            coord = ''
            i = i+1
            continue
        coord = coord+road[i]
        i = i+1

    if tunnel == 'yes':
        color = 'red'
    else:
        color = 'green'
    folium.PolyLine(points, color=color, weight=2, opacity=1).add_to(m)
    print ('# of iter: ', num)
    return points
    
    
# SELECT id, source, wn.node_id,
# MIN(DISTANCE(osm.geometry, ST_POINT( -83.702175,42.297199))) as dist 
# FROM OSM_AA_POLYLINES_NEW AS osm,
# ways_nodes AS wn
# WHERE osm.id=wn.way_id

'''Find the number of points and the points coordinate of a road.'''
# SELECT AsText(geometry), NumPoints(geometry),X(PointN(geometry,2))
# FROM OSM_AA_POLYLINES_NEW as osm
# WHERE osm.id='4411729'

'Find the number of points and the points coordinate of a road.'

In [96]:
if __name__ == "__main__":
    db_name = "data/AL_osm.db"
    SQL = sql(db_name)
    tunnel_types = SQL.load_tunnel()
    #SQL.count_surfaces()
    #SQL.count_highway()

(u'', u'LINESTRING(-83.621341 42.512915, -83.621322 42.513002, -83.621292 42.513117, -83.621309 42.513283)')


In [86]:
if __name__ == "__main__":
    db_name = "data/OSM_AA.osm.db"
    SQL = sql(db_name)
    
    long = -83.702175
    lat = 42.297199
    location = [long, lat]
    nearest_road = SQL.reorder_roads(location)
    print (nearest_road['highway'], nearest_road['maxspeed'])

residential 13.4112


In [100]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

m = folium.Map(location=[43.5, -84.5], zoom_start=7, tiles="cartodbpositron")

for i, road in enumerate(tunnel_types['geometry']):
    string2points(road, tunnel_types['tunnel'][i], i, m)
    if i >= 2000:
        break
# Parallel(n_jobs=num_cores)\
#         (delayed(string2points)(road, tunnel_types['tunnel'][i], i, m)\
#         for i, road in enumerate(tunnel_types['geometry']))
m
# for i, road in enumerate(tunnel_types['geometry']):
#     points = SQL.string2points(road, tunnel_types['tunnel'][i])
#     print (i)


('# of iter: ', 0)
('# of iter: ', 1)
('# of iter: ', 2)
('# of iter: ', 3)
('# of iter: ', 4)
('# of iter: ', 5)
('# of iter: ', 6)
('# of iter: ', 7)
('# of iter: ', 8)
('# of iter: ', 9)
('# of iter: ', 10)
('# of iter: ', 11)
('# of iter: ', 12)
('# of iter: ', 13)
('# of iter: ', 14)
('# of iter: ', 15)
('# of iter: ', 16)
('# of iter: ', 17)
('# of iter: ', 18)
('# of iter: ', 19)
('# of iter: ', 20)
('# of iter: ', 21)
('# of iter: ', 22)
('# of iter: ', 23)
('# of iter: ', 24)
('# of iter: ', 25)
('# of iter: ', 26)
('# of iter: ', 27)
('# of iter: ', 28)
('# of iter: ', 29)
('# of iter: ', 30)
('# of iter: ', 31)
('# of iter: ', 32)
('# of iter: ', 33)
('# of iter: ', 34)
('# of iter: ', 35)
('# of iter: ', 36)
('# of iter: ', 37)
('# of iter: ', 38)
('# of iter: ', 39)
('# of iter: ', 40)
('# of iter: ', 41)
('# of iter: ', 42)
('# of iter: ', 43)
('# of iter: ', 44)
('# of iter: ', 45)
('# of iter: ', 46)
('# of iter: ', 47)
('# of iter: ', 48)
('# of iter: ', 49)
('# of ite

('# of iter: ', 405)
('# of iter: ', 406)
('# of iter: ', 407)
('# of iter: ', 408)
('# of iter: ', 409)
('# of iter: ', 410)
('# of iter: ', 411)
('# of iter: ', 412)
('# of iter: ', 413)
('# of iter: ', 414)
('# of iter: ', 415)
('# of iter: ', 416)
('# of iter: ', 417)
('# of iter: ', 418)
('# of iter: ', 419)
('# of iter: ', 420)
('# of iter: ', 421)
('# of iter: ', 422)
('# of iter: ', 423)
('# of iter: ', 424)
('# of iter: ', 425)
('# of iter: ', 426)
('# of iter: ', 427)
('# of iter: ', 428)
('# of iter: ', 429)
('# of iter: ', 430)
('# of iter: ', 431)
('# of iter: ', 432)
('# of iter: ', 433)
('# of iter: ', 434)
('# of iter: ', 435)
('# of iter: ', 436)
('# of iter: ', 437)
('# of iter: ', 438)
('# of iter: ', 439)
('# of iter: ', 440)
('# of iter: ', 441)
('# of iter: ', 442)
('# of iter: ', 443)
('# of iter: ', 444)
('# of iter: ', 445)
('# of iter: ', 446)
('# of iter: ', 447)
('# of iter: ', 448)
('# of iter: ', 449)
('# of iter: ', 450)
('# of iter: ', 451)
('# of iter: 

('# of iter: ', 808)
('# of iter: ', 809)
('# of iter: ', 810)
('# of iter: ', 811)
('# of iter: ', 812)
('# of iter: ', 813)
('# of iter: ', 814)
('# of iter: ', 815)
('# of iter: ', 816)
('# of iter: ', 817)
('# of iter: ', 818)
('# of iter: ', 819)
('# of iter: ', 820)
('# of iter: ', 821)
('# of iter: ', 822)
('# of iter: ', 823)
('# of iter: ', 824)
('# of iter: ', 825)
('# of iter: ', 826)
('# of iter: ', 827)
('# of iter: ', 828)
('# of iter: ', 829)
('# of iter: ', 830)
('# of iter: ', 831)
('# of iter: ', 832)
('# of iter: ', 833)
('# of iter: ', 834)
('# of iter: ', 835)
('# of iter: ', 836)
('# of iter: ', 837)
('# of iter: ', 838)
('# of iter: ', 839)
('# of iter: ', 840)
('# of iter: ', 841)
('# of iter: ', 842)
('# of iter: ', 843)
('# of iter: ', 844)
('# of iter: ', 845)
('# of iter: ', 846)
('# of iter: ', 847)
('# of iter: ', 848)
('# of iter: ', 849)
('# of iter: ', 850)
('# of iter: ', 851)
('# of iter: ', 852)
('# of iter: ', 853)
('# of iter: ', 854)
('# of iter: 

('# of iter: ', 1214)
('# of iter: ', 1215)
('# of iter: ', 1216)
('# of iter: ', 1217)
('# of iter: ', 1218)
('# of iter: ', 1219)
('# of iter: ', 1220)
('# of iter: ', 1221)
('# of iter: ', 1222)
('# of iter: ', 1223)
('# of iter: ', 1224)
('# of iter: ', 1225)
('# of iter: ', 1226)
('# of iter: ', 1227)
('# of iter: ', 1228)
('# of iter: ', 1229)
('# of iter: ', 1230)
('# of iter: ', 1231)
('# of iter: ', 1232)
('# of iter: ', 1233)
('# of iter: ', 1234)
('# of iter: ', 1235)
('# of iter: ', 1236)
('# of iter: ', 1237)
('# of iter: ', 1238)
('# of iter: ', 1239)
('# of iter: ', 1240)
('# of iter: ', 1241)
('# of iter: ', 1242)
('# of iter: ', 1243)
('# of iter: ', 1244)
('# of iter: ', 1245)
('# of iter: ', 1246)
('# of iter: ', 1247)
('# of iter: ', 1248)
('# of iter: ', 1249)
('# of iter: ', 1250)
('# of iter: ', 1251)
('# of iter: ', 1252)
('# of iter: ', 1253)
('# of iter: ', 1254)
('# of iter: ', 1255)
('# of iter: ', 1256)
('# of iter: ', 1257)
('# of iter: ', 1258)
('# of ite

('# of iter: ', 1624)
('# of iter: ', 1625)
('# of iter: ', 1626)
('# of iter: ', 1627)
('# of iter: ', 1628)
('# of iter: ', 1629)
('# of iter: ', 1630)
('# of iter: ', 1631)
('# of iter: ', 1632)
('# of iter: ', 1633)
('# of iter: ', 1634)
('# of iter: ', 1635)
('# of iter: ', 1636)
('# of iter: ', 1637)
('# of iter: ', 1638)
('# of iter: ', 1639)
('# of iter: ', 1640)
('# of iter: ', 1641)
('# of iter: ', 1642)
('# of iter: ', 1643)
('# of iter: ', 1644)
('# of iter: ', 1645)
('# of iter: ', 1646)
('# of iter: ', 1647)
('# of iter: ', 1648)
('# of iter: ', 1649)
('# of iter: ', 1650)
('# of iter: ', 1651)
('# of iter: ', 1652)
('# of iter: ', 1653)
('# of iter: ', 1654)
('# of iter: ', 1655)
('# of iter: ', 1656)
('# of iter: ', 1657)
('# of iter: ', 1658)
('# of iter: ', 1659)
('# of iter: ', 1660)
('# of iter: ', 1661)
('# of iter: ', 1662)
('# of iter: ', 1663)
('# of iter: ', 1664)
('# of iter: ', 1665)
('# of iter: ', 1666)
('# of iter: ', 1667)
('# of iter: ', 1668)
('# of ite